In [15]:
import glob
from glob import glob
import shutil
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading

global processlist, df
conn = False
con = None

filepath = r'DataAndDuplicateReportAutomation'
try:
    
    logging.basicConfig(filename = 'RRMDuplicateExceptionsFinalSection.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    


    def check_files():
        global processlist
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def convert_to_CSV(doc):
        #global df
        pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name='Daily data report').to_csv(f'{filepath}'+'/'+'DataAndDuplicateReport' + '_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.csv', encoding = 'utf-8', index = False)
        shutil.move(f'{filepath}'+'/'+f'{doc}', 'DataAndDuplicateReportAutomation/Archive/xlsx/'+f'{doc}')
        #for f in glob('DataAndDuplicateReportAutomation/DataAndDuplicateReport*.csv'):
            #df = pd.read_csv(f)
        #display(df)
        return #df
    
    def process_CSV():
        global processlist, df
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport") and file.endswith(".csv"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def move_processed():
        global processedlist
        processedlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DDRProcessed") and file.endswith(".csv"):
                print(f'\n{file}')
                processedlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DDRProcessed"):
                print('incorrect file name is present.')
        if processedlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processedlist += ['None']
        else:
            pass
        return processedlist

    def duplication_extraction():
    
        global df1pre, df1, df2, df3_nofilter
        
        pd.set_option('display.max_columns', None)
    
        df1pre = pd.read_csv(f'DataAndDuplicateReportAutomation/{processlist[0]}')
        
        df1 = df1pre.sort_values(by=['PersonPartyID','CreatedDatetime'], ascending = True)

        today = date.today()
        first = today.replace(day=1)
        last_month = first - timedelta(days=1)
        tenth_of_current_month = today.replace(day=10)
        first_of_previous_month = last_month.replace(day=1)
        next_month = today.replace(day=28) + timedelta(days=4)
        next_month = next_month - timedelta(days=next_month.day)

        df1['CreatedDatetime']= pd.to_datetime(df1['CreatedDatetime']).dt.date
        df1['EffectiveDate'] = pd.to_datetime(df1['EffectiveDate']).dt.date
        df1.loc[(df1['ExpiryDate'].isna()), 'ExpiryDate'] = '1/1/1990'
        df1['ExpiryDate'] = pd.to_datetime(df1['ExpiryDate']).dt.date
        df1['LastModifiedDatetime'] = pd.to_datetime(df1['LastModifiedDatetime']).dt.date

        df1 = df1.where(pd.notnull(df1), None)

        df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Previous'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['CreatedDatetime'].transform(max) == df1['CreatedDatetime']), 'CreatedDatetime_status'] = 'Latest'

        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest'
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Current'
        df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & Min Exp Date Error'
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Latest CD but not Max Exp Date Error'
        df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & ((df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) == df1['ExpiryDate']) & ~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(min) == df1['ExpiryDate'])), 'ExpiryDate_status'] = 'Previous CD but Max & not Min Exp Date Error'
        df1.loc[~(df1['CreatedDatetime_status'] == 'Latest') & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['ExpiryDate'].transform(max) != df1['ExpiryDate']), 'ExpiryDate_status'] = 'Previous CD and not Max Exp Date Error'
        df1.loc[pd.to_datetime(df1['ExpiryDate']) == '1990-01-01', 'ExpiryDate_status'] += ' & Null Placeholder'
        df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'

        df1.loc[~(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Previous EffectiveDate'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']), 'EffectiveDate_status'] = 'Current EffectiveDate'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['LastModifiedDatetime'].transform(max) == df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Latest MD'
        df1.loc[(df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max) == df1['EffectiveDate']) & (df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['LastModifiedDatetime'].transform(max) != df1['LastModifiedDatetime']), 'EffectiveDate_status'] += ' & Previous MD'
        
        df1.loc[(df1['EffectiveDate']) < (df1['CreatedDatetime']), 'EffectiveDate_status'] += ' & Eff Date less than CD Anomaly'
        df1.loc[(((df1['EffectiveDate']) >= (df1['ExpiryDate'])) & ~(pd.to_datetime(df1['ExpiryDate']) == '1990-01-01')), 'EffectiveDate_status'] += ' & Eff Date greater than/Equal to Exp Date Error'

        todayv2 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%Y-%m-%d')
        todayv3 = datetime.strptime(str(today), '%Y-%m-%d').strftime('%m-%d')
        last_monthv2 = datetime.strptime(str(last_month), '%Y-%m-%d').strftime('%Y-%m-%d')
        df1['ExpiryDateFirstOfPreviousMonthStr'] = ((df1['ExpiryDate']) - relativedelta(months=1)).astype('str')
        df1['ExpiryDateMaxEffDateMinus1'] = df1.groupby(['PersonPartyID','LegalEntityPartyID','rateType','RateBasis'])['EffectiveDate'].transform(max)
        df1['ExpiryDateMaxEffDateMinus1'] = ((df1['ExpiryDateMaxEffDateMinus1']) - relativedelta(days=1)).astype('str')
        df1['Update_Statement'] = 'update resourcepersonrate set'
        df1.loc[df1['ExpiryDate_status'].str.contains('Expired') , 'Update_Statement'] += " expirydate = '1990-01-01'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD & Eff Date less than CD Anomaly'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date greater than/Equal to Exp Date Error'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & not Min Exp Date Error') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD and not Max Exp Date Error') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1['ExpiryDate_status'] == 'Current & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate & Eff Date less than CD Anomaly') & df1['InternalpersonCategoryname'].str.contains('Practice Support'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1['ExpiryDate_status'] == 'Current & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Previous MD & Eff Date less than CD Anomaly') & df1['InternalpersonCategoryname'].str.contains('Practice Support'), 'Update_Statement'] += f" expirydate = '"+df1['ExpiryDateMaxEffDateMinus1']+"'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Latest') & (df1['ExpiryDate_status'] == 'Current & Null Placeholder') & (df1['EffectiveDate_status'] == 'Previous EffectiveDate'), 'Update_Statement'] += f" expirydate = '{last_monthv2}'"
        df1.loc[(df1['CreatedDatetime_status'] == 'Previous') & (df1['ExpiryDate_status'] == 'Previous CD but Max & Min Exp Date Error & Null Placeholder') & (df1['EffectiveDate_status'] == 'Current EffectiveDate & Latest MD') & df1['ResourceRole'].str.contains('Specialist'), 'Update_Statement'] = ''
        df1.loc[df1['Update_Statement'] == 'update resourcepersonrate set' , 'Update_Statement'] = ''
        df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", lastmodifieddatetime = '{todayv2}'"
        df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f", comments = comments +' | edit from {todayv3} correction automation'"
        df1.loc[df1['Update_Statement'] != '' , 'Update_Statement'] += f" where resourcepersonratesk = '"+df1['Resourcepersonratesk'].astype('str')+"'"

        df2 = df1[['PersonPartyID','LegalEntityPartyID','rateType','EffectiveDate','ExpiryDate','CreatedDatetime','LastModifiedDatetime','RateBasis','CreatedDatetime_status','ExpiryDate_status','EffectiveDate_status','Update_Statement']]
        shutil.move(f'{filepath}'+'/'+f'{processlist[0]}', 'DataAndDuplicateReportAutomation/Temp/'+f'{processlist[0]}')
        df2.to_csv(f'{filepath}'+'/'+'DDRProcessed' + '_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.csv', encoding = 'utf-8', index = False)
        #display(df2)

        return

    check_files()
    if 'None' not in processlist:
        print('\ndocuments available for processing\n')
        print(processlist)
        for doc in enumerate(processlist):
            try:
                gstart_time = datetime.now()
                convert_to_CSV(doc[1])
                gend_time = datetime.now()
                process_CSV()
                duplication_extraction()
                move_processed()
                print("")
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
            except Exception as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
            for i in range(10, -1, -1):
                    print(f"{i} seconds until next document is processed", end = "\r")
                    ti.sleep(1)
    else:
        print('documents not available for processing')
        pass
    
    if 'None' not in processedlist:
        for file in processedlist:
            shutil.move(f'{filepath}'+'/'+f'{processedlist[0]}', 'DataAndDuplicateReportAutomation/Archive/csv/'+f'{processedlist[0]}')
            move_processed()
        print('Processing Complete')
    else:
        pass

#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    print("All errors identified.")
    if conn == True:
        close_connection()
        cur.close()

Analysis
Archive
DataAndDuplicateReport_110822.xlsx

DataAndDuplicateReport_110822.xlsx
DataAndDuplicateReport_110922.xlsx

DataAndDuplicateReport_110922.xlsx
Temp

documents available for processing

['DataAndDuplicateReport_110822.xlsx', 'DataAndDuplicateReport_110922.xlsx']
Analysis
Archive
DataAndDuplicateReport_110922.xlsx
incorrect file name is present.
DataAndDuplicateReport_2022-12-30_13-48-41.csv

DataAndDuplicateReport_2022-12-30_13-48-41.csv
Temp


<ipython-input-15-d67b2ab4730a>:150: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'


Analysis
Archive
DataAndDuplicateReport_110922.xlsx
incorrect file name is present.
DDRProcessed_2022-12-30_13-52-13.csv

DDRProcessed_2022-12-30_13-52-13.csv
Temp


Duration: 0:00:46.084849
Analysiss until next document is processedd
Archive
DataAndDuplicateReport_2022-12-30_13-53-11.csv

DataAndDuplicateReport_2022-12-30_13-53-11.csv
Temp


<ipython-input-15-d67b2ab4730a>:150: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  df1.loc[(pd.to_datetime(df1['ExpiryDate']) != '1990-01-01') & (pd.to_datetime(date.today()) >= df1['ExpiryDate']), 'ExpiryDate_status'] += ' & Expired'


Analysis
Archive
DDRProcessed_2022-12-30_13-56-19.csv

DDRProcessed_2022-12-30_13-56-19.csv
Temp


Duration: 0:00:47.660962
[Errno 2] No such file or directory: 'DataAndDuplicateReportAutomation/DDRProcessed_2022-12-30_13-56-19.csv'
All errors identified.


In [1]:
pip install xlrd==2.0.1

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.2.0
    Uninstalling xlrd-1.2.0:
      Successfully uninstalled xlrd-1.2.0

